In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\chicken_Disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\chicken_Disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

@dataclass(frozen= True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [6]:
from CNNCDC.constants import*
from CNNCDC.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        Initializes the ConfigurationManager by loading configuration and
        parameters from YAML files.
        """
        # Load the main configuration file
        self.config = read_yaml(config_filepath)
        # Load the model hyperparameters
        self.params = read_yaml(params_filepath)

        # Create the root artifacts directory if it doesn't exist
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config= self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.Unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.training_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate= params.LEARNING_RATE
        )

        return training_config

In [8]:
import time

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
from pathlib import Path
import tensorflow as tf
import time

In [10]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _creat_tb_callbacks(self):
        timestamp= time.strftime("%Y-%m-%d-%H-%S")
        tb_runing_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at_(timestamp)"
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_runing_log_dir)
    
    @property
    def _creat_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    

    def get_tb_ckpt_callbacks(self):
        return[
            self._creat_tb_callbacks,
            self._creat_ckpt_callbacks
        ]

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config


    def get_base_model(self):
        self.model= tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def compile_model(self):
        # The .compile() method configures the model for training.
        self.model.compile(
            # The optimizer updates the model based on the data and the loss function.
            # Here, we use Stochastic Gradient Descent (SGD) with the learning rate from the config.
            optimizer = tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            # The loss function measures how accurate the model is.
            # CategoricalCrossentropy is suitable for multi-class classification problems.
            loss = tf.keras.losses.CategoricalCrossentropy(),
            # The metrics list is used to monitor the training and testing steps.
            # "accuracy" is a common metric to track model performance.
            metrics=["accuracy"]
            )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'biliner'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )


        if self.config.params_is_augmentation:
            train_datageneration = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range =40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs

            )

        else:
            train_datageneration = valid_datagenerator
        self.train_generator = train_datageneration.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callbacks_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callbacks_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks( config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()


    training_config = config.get_training_config()
    training = Training(config=training_config)
    
    # 1. Load the model's architecture and weights.
    training.get_base_model() 
    
    # 2. Re-compile the model. This is the crucial step.
    training.compile_model() 
    
    # 3. Set up the data generators.
    training.train_valid_generator()
    
    # 4. Start the training process.
    training.train(
        callbacks_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-09-26 03:11:08,257: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-26 03:11:08,260: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-26 03:11:08,262: INFO: common: created directory at: artifacts]
[2025-09-26 03:11:08,264: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-09-26 03:11:08,266: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-09-26 03:11:08,267: INFO: common: created directory at: artifacts\training]
[2025-09-26 03:11:08,660: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


d:\Data_science\Projects\chicken_Disease_classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3121 - loss: 33.0431

d:\Data_science\Projects\chicken_Disease_classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


[2025-09-26 03:12:25,778: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
19/19 ━━━━━━━━━━━━━━━━━━━━ 73s 4s/step - accuracy: 0.5000 - loss: 26.9738 - val_accuracy: 0.3906 - val_loss: 34.0890
[2025-09-26 03:12:25,951: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
